### RAGを使ってチャット
1. Model読み込み
2. PromptTemplateの設定
3. FAISSのvector dataを取得
4. Chatの実装

In [1]:
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import (
    AzureOpenAIEmbeddings,
    OpenAIEmbeddings,
    AzureChatOpenAI,
    ChatOpenAI
)

from langchain_core.messages import (
    HumanMessage, 
    AIMessage,
    SystemMessage
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from dotenv import load_dotenv
load_dotenv('../.env')

True

#### 1. Model読み込み

In [2]:
# emmbeddingsのモデルを取得
embeddings = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="embedding",
        openai_api_version="2024-06-01"
    )
elif os.getenv('OPENAI_API_KEY') != "":
    # OpenAIの場合
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
else:
    print("APIKeyの設定を確認してください")

# chatのモデルを取得
model = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    model = AzureChatOpenAI(
        azure_deployment="chat",
        openai_api_version="2024-06-01"
    )
elif os.getenv('OPENAI_API_KEY') != "":
    # OpenAIの場合
    model = ChatOpenAI(model="gpt-4")
else:
    print("APIKeyの設定を確認してください")

#### 2.PromptTemplateの設定

##### 2-1. コンテキストから回答するプロンプトとChain

In [3]:
system_prompt = (
    "あなたは質問対応のアシスタントです。"
    "質問に答えるために、検索された文脈の以下の部分を使用してください。"
    "答えがわからない場合は、わからないと答えてください。"
    "回答は3文以内で簡潔にしてください。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model

##### 2-2. 質問を要約するプロンプトとChain

In [9]:
from langchain_core.output_parsers import StrOutputParser

contextualize_q_system_prompt = (
    "あなたは、AIでチャットの質問を作り直すように求められています。"
    "チャット履歴と最新のユーザーメッセージがあり、そのメッセージは"
    "チャット履歴のコンテキストを参照している質問である可能性があります。"
    "チャット履歴がなくても、理解できる独立した質問を作成してください。"
    "絶対に、質問に答えないでください。"
    "質問は、「教えてください。」「どういうことですか？」などAIに投げかける質問にしてください。"
    "メッセージが質問であれば、作り直してください。"
    "「ありがとう」などメッセージが質問ではない場合は、メッセージを作り直さず戻してください。"
    "\n\n"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
contextualize_chain = contextualize_q_prompt | model | StrOutputParser()

#### 3. FAISSのvector dataを取得

In [4]:
vectorstore = FAISS.load_local("./db", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

#### 4. Chatの実装
チャットの実装では下記図のように2段階の処理をしていきます  
<img src="./../docs/asset/image3.png" width="600px">  

<a href="https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/" target=_blank>Langchain Doc</a>

##### 4-1. 二つの質問をしてみる
 まず、1つ目のステップを確認していきます。  
 なぜ1つ目の処理をする必要があるのかをみるために、2つの質問をします

In [5]:
messages = []
msg1 = "LLMはどんな訓練方法がありますか？"
relavant_docs = retriever.invoke(msg1, k=3)
chain.invoke({"chat_history": messages, "context": relavant_docs, "input": msg1})

AIMessage(content='LLMの訓練方法には、教師あり学習、強化学習（RLHF）、そしてファインチューニングが含まれます。例えば、OpenAIのInstructGPTプロトコルは、人間が作成したプロンプトと応答の組からなるデータセットを用いた教師ありファインチューニングと、その後の人間のフィードバックを用いた強化学習を行います。', response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 4038, 'total_tokens': 4176}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_e49e4201a9', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-0b89b3f3-2a43-4f

In [6]:
messages = [
    HumanMessage(content="LLMはどんな訓練方法がありますか？"),
    AIMessage(content="LLMの訓練方法には、教師ありファインチューニング、強化学習、ツールのファインチューニング、検索拡張生成（RAG）などがあります。"),
]
msg2 = "1つ目について教えてください。"
relavant_docs = retriever.invoke(msg2, k=3)
chain.invoke({"chat_history": messages, "context": relavant_docs, "input": msg2})

AIMessage(content='申し訳ありませんが、具体的な教師ありファインチューニングについての説明は提供されていません。このトピックについては、追加情報が必要です。', response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 4347, 'total_tokens': 4413}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_e49e4201a9', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-09a8c5e8-3d09-4c6a-9c80-33118c575435-0', usage_metadata={'input_tokens': 4347, 'output_tokens': 66, 't

「"1つ目について教えてください。"」からだと正確なドキュメントが取得できない

In [7]:
# 類似文書を表示
for doc in relavant_docs:
    print(doc)
    print("-----")

page_content='ス埋めパズル、ヒングリッシュ（ヒンディー語と英語の混成語）の段落内
の不快な内容の特定、およびスワヒリ語のことわざに相当する英語の⽣
成などがある[16]。
Schaeffer らは、創発的な能⼒は予測不可能な形で獲得されるのではなく、滑らかなスケーリング則に従って
予測通りに獲得されると主張している[17]。著者らは、 LLM が多肢選択問題を解く統計的トイモデルを検討
し、他の種類のタスクを考慮して修正されたこの統計モデルが、これらのタスクにも適⽤できることを⽰し
た。
ここで、 をパラメータ数、  をモデルの性能と する。
のとき、  は指数曲線 （1でプラトーに達する前）となり、創発
のように⾒え る。
 のとき、  のプロットは直線（0でプラトーに達 する前）
となり、創発には⾒えない。創発的能⼒ のとき、  はステッ プ関数 となり、創発の
ように⾒え る。
⼤規模⾔語モデルの基本的な考え⽅は、単純で反復的なアーキテクチャを持つランダムな重みを持つニュー
ラルネットワークを出発点とし、⼤規模な⾔語コー パスで訓練することである。
この最も初期の例のひとつがエルマンネットワークで[18]、「⽝が男を追いかける」のような単純な⽂でリカ
レントネットワークを訓練した。訓練したネットワークは、各単語をベクトル（内部表現）に変換した。次
にこれらのベクトルを接近度によって⽊構造にクラスタリングした。その結果、ツリーはある構造を⽰すこ
とがわかった。動詞と名詞はそれぞれ別の⼤きなクラスターに属していた。名詞のクラスター内には、無⽣
物（ inanimates ）と⽣物（ animates ）の 2 つの⼩さなクラ スターがある、などである。
別の⽅法として、⾃然⾔語理解を記号プログラムによってコンピュータにプログラムする論理 AIがあった。
この⽅法は 1990 年代まで主流であった。単純な機構と⼤規模なコーパスによって⾃然⾔語を学習するという
着想は 1950 年代に始まったが、商業的に最初に成功したのは、統計的機械翻訳のためのIBMアライメントモ
デル（1990年代）であった。
初期の「⼤規模」⾔語モデルは、⻑期・短期記憶（LSTM、 1997 年）などのリカレントアーキテクチャを使'
-----
page_content='返し触れ

##### 4-2. 履歴から質問を作り直す
それを解決するための方法が、1つ目のステップになります。  
やっていることは、履歴から質問を作り出しています

In [10]:
# sample1
messages = [
    HumanMessage(content="LLMはどんな訓練方法がありますか？"),
    AIMessage(content="LLMの訓練方法には、教師ありファインチューニング、強化学習、ツールのファインチューニング、検索拡張生成（RAG）などがあります。"),
]
new_msg = contextualize_chain.invoke({"chat_history": messages, "input": "1つ目についておしえてください"})
print(new_msg)


教師ありファインチューニングについて詳しく説明してください。


In [12]:
# sample2
messages = [
    HumanMessage(content="Pythonについて教えて"),
    AIMessage(content="Pythonは、1991年にオランダ人プログラマーのグイド・ヴァンロッサム氏によって開発されたオープンソース形式のプログラミング言語です。主に人工知能の開発、データ処理、Webアプリケーションの開発など幅広い用途で使用されています。また、初心者にも学びやすい言語とされ、豊富なライブラリが存在するのが特徴です。"),
    HumanMessage(content="どんな勉強方法がありますか？"),
    AIMessage(content="Pythonを学ぶ方法にはいくつかの選択肢があります。一つは書籍を使用して学習する方法で、これは自分で内容を選びながら学ぶことが推奨されます。また、無料の動画サイトやYouTubeでPythonに関する教育動画を視聴する方法もあります。さらに、Progateやドットインストール、Udemyなどの有料の学習サイトを利用する方法も一般的です。それぞれの方法にはメリットとデメリットがあるため、個々の学習スタイルや目的に合わせて選択することが重要です。"),
]
new_msg = contextualize_chain.invoke({"chat_history": messages, "input": "1つ目について教えてください"})
print(new_msg)


Pythonの勉強方法として、書籍を使用する方法について詳しく教えてください。


#### 4-3. チャットの実装
 ここでは公式ドキュメントの方法ではなく、理解のしやすさとバージョン更新の影響を減らすために  
 Langchainの基本的な（変更が今後すくなさそうな）処理で実装しています

In [13]:
messages_sample = [
    "LLMはどんな訓練方法がありますか？",
    "2つ目についておしえてください"
]
messages = []

In [14]:
for msg in messages_sample:
    # 質問を修正する
    new_msg = contextualize_chain.invoke({"chat_history": messages, "input": msg})
    print("Human:", msg, "> ", new_msg)
    # 関連ドキュメントを取得
    relavant_docs = retriever.invoke(new_msg, k=3)
    # 質問に回答する
    response = chain.invoke({"chat_history": messages, "context": relavant_docs, "input": msg})
    print("AI:", response.content)
    # メッセージを保存
    messages.extend([
        HumanMessage(content=msg), # 作り直したほうではなく、ユーザー入力の方にする
        AIMessage(content=response.content)
    ])

Human: LLMはどんな訓練方法がありますか？ >  LLM（大規模言語モデル）の訓練方法にはどのようなものがありますか？
AI: LLM（大規模言語モデル）は、膨大なラベルなしテキストを使用して自己教師あり学習または半教師あり学習によって訓練が行われます。
Human: 2つ目についておしえてください >  LLMの半教師あり学習について詳しく教えてください。
AI: 半教師あり学習では、ラベル付きデータとラベルなしデータの両方を使用してモデルを訓練します。これにより、ラベル付きデータが少ない場合でも効果的に学習を進めることができます。


In [15]:
messages

[HumanMessage(content='LLMはどんな訓練方法がありますか？'),
 AIMessage(content='LLM（大規模言語モデル）は、膨大なラベルなしテキストを使用して自己教師あり学習または半教師あり学習によって訓練が行われます。'),
 HumanMessage(content='2つ目についておしえてください'),
 AIMessage(content='半教師あり学習では、ラベル付きデータとラベルなしデータの両方を使用してモデルを訓練します。これにより、ラベル付きデータが少ない場合でも効果的に学習を進めることができます。')]

#### 4-5. チャットの実装 Streaming

In [16]:
for msg in messages_sample:
    # 質問を修正する
    new_msg = contextualize_chain.invoke({"chat_history": messages, "input": msg})
    print("Human:", msg, "> ", new_msg)
    # 関連ドキュメントを取得
    relavant_docs = retriever.invoke(new_msg, k=3)
    # 質問に回答する Streaming
    full_response = ""
    for r in chain.stream({"chat_history": messages, "context": relavant_docs, "input": msg}):
        full_response+=r.content
        print(r.content, end="|")    
    print("/n")
    messages.extend([
        HumanMessage(content=msg),
        AIMessage(content=full_response)
    ])

Human: LLMはどんな訓練方法がありますか？ >  LLMの訓練方法にはどのようなものがありますか？
|LL|M|の|訓|練|方法|に|は|、|教|師|あり|学|習|、|自|己|教|師|あり|学|習|、|強|化|学|習|（|RL|HF|）、|カ|リ|キ|ュ|ラ|ム|学|習|な|ど|が|あり|ます|。|これ|ら|の|方法|を|通|じ|て|、|モ|デ|ル|は|膨|大|な|テ|キ|スト|デ|ータ|から|パ|タ|ー|ン|を|学|習|し|、|言|語|的|タ|ス|ク|を|実|行|する|能|力|を|身|に|つ|け|ます|。||/n
Human: 2つ目についておしえてください >  自己教師あり学習とは、どのような訓練方法ですか？
|自|己|教|師|あり|学|習|では|、|ラ|ベ|ル|付|き|デ|ータ|を|必|要|と|せ|ず|、|モ|デ|ル|自|身|が|生成|した|デ|ータ|を|使用|して|訓|練|を|行|います|。|例|え|ば|、|文|から|一|部|の|単|語|を|隠|し|、|モ|デ|ル|に|その|隠|され|た|単|語|を|予|測|さ|せ|る|こ|と|で|、|言|語|の|理|解|を|深|め|る|訓|練|が|行|わ|れ|ます|。|この|方法|は|大|量|の|未|ラ|ベ|ル|の|テ|キ|スト|デ|ータ|を|活|用|で|き|る|た|め|、|実|世|界|の|ア|プ|リ|ケ|ーシ|ョ|ン|に|適|して|います|。||/n
